In [ ]:
# import yfinance as yf
# # Define the time range
# aapl = yf.Ticker("AAPL")
# # Fetch historical data since 2006
# aapl_data = aapl.history(start="2014-10-29", end=None)
# aapl_data = aapl_data[['Open', 'High', 'Low', 'Close', 'Volume']]
# # Show the first few rows
# print(aapl_data.head())
# # Save it to a CSV file
# aapl_data.to_csv("aapl_data.csv")

                                Open       High        Low      Close  \
Date                                                                    
2014-10-29 00:00:00-04:00  23.647813  23.807461  23.583511  23.800808   
2014-10-30 00:00:00-04:00  23.716542  23.803018  23.481506  23.720978   
2014-10-31 00:00:00-04:00  23.949369  23.956021  23.771982  23.947151   
2014-11-03 00:00:00-05:00  23.995937  24.457142  23.949373  24.257582   
2014-11-04 00:00:00-05:00  24.248704  24.277529  23.885063  24.080187   

                              Volume  
Date                                  
2014-10-29 00:00:00-04:00  210751600  
2014-10-30 00:00:00-04:00  162619200  
2014-10-31 00:00:00-04:00  178557200  
2014-11-03 00:00:00-05:00  209130400  
2014-11-04 00:00:00-05:00  166297600  


In [4]:
import pandas as pd
import numpy as np
data = pd.read_csv('aapl_data.csv')

#Estandarizamos fecha y precio
data['Close'] = data['Close'].astype(float)  
data['Date'] = pd.to_datetime(data['Date'])  

# Calculamos métricas

# SMA_Ratio: Relación entre precio de cierre y promedio móvil simple (20 días)
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_Ratio'] = data['Close'] / data['SMA_20']

# MACD_hist: Histograma de MACD
ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema_12 - ema_26
data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['MACD_hist'] = data['MACD'] - data['Signal_Line']

# RSI (Relative Strength Index, 14 días)
delta = data['Close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI'] = 100 - (100 / (1 + rs))

# SO_diff: Diferencia del oscilador estocástico
low_14 = data['Close'].rolling(window=14).min()
high_14 = data['Close'].rolling(window=14).max()
data['%K'] = (data['Close'] - low_14) / (high_14 - low_14) * 100
data['%D'] = data['%K'].rolling(window=3).mean()
data['SO_diff'] = data['%K'] - data['%D']

# Returns: Retorno logarítmico diario
data['Returns'] = np.log(data['Close'] / data['Close'].shift(1))

# Exportar o continuar trabajando con el DataFrame resultante
data.to_csv('aapl_data_with_metrics.csv', index=False)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5064\2927819463.py:7: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['Date'] = pd.to_datetime(data['Date'])
